<a href="https://colab.research.google.com/github/yeray142/AIML_project/blob/master/notebooks/SVC_with_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML4AI | SVM with "CODE" column 

### Set up for Project

##### Library

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 6.0 MB/s 
     |████████████████████████████████| 209 kB 46.4 MB/s 
     |████████████████████████████████| 81 kB 5.3 MB/s 
     |████████████████████████████████| 78 kB 2.4 MB/s 
     |████████████████████████████████| 50 kB 2.1 MB/s 
     |████████████████████████████████| 147 kB 12.6 MB/s 
     |████████████████████████████████| 112 kB 3.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=f17d033c2d6ed62781191edec13ffb93220350b1830f9cc68f87f39b105868de
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [3]:
# Modeling
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score, KFold,StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from sklearn.svm import SVC
from sklearn.metrics import log_loss

In [4]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

#### Data Load

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df_train = pd.read_csv("/content/drive/MyDrive/ML4AI/processed_train_code.csv")
df_test = pd.read_csv("/content/drive/MyDrive/ML4AI/processed_test_code.csv")

In [7]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_test = df_test.drop(['index'],axis =1)

## Data EDA

#### **Basic EDA**

In [8]:
df_train.head()

,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,work_phone,home_phone,...,Age,birth_month,birth_week,ages_employed,employ_month,employ_week,ages_unemployed,unemploy_month,unemploy_week,income_family
0,F,N,N,202500.0,Commercial associate,Higher education,Married,Municipal apartment,0,0,...,38,7.0,1.0,12,0.0,0.0,25,6.0,0.0,101250.0
1,F,N,Y,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,0,0,...,31,7.0,1.0,4,3.0,0.0,26,4.0,1.0,82500.0
2,M,Y,Y,450000.0,Working,Higher education,Married,House / apartment,0,1,...,52,0.0,2.0,12,3.0,1.0,40,8.0,1.0,225000.0
3,F,N,Y,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,0,1,...,41,10.0,3.0,5,9.0,2.0,35,1.0,0.0,101250.0
4,F,Y,Y,157500.0,State servant,Higher education,Married,House / apartment,0,0,...,41,9.0,0.0,5,10.0,0.0,35,11.0,3.0,78750.0


In [9]:
df_test.head()

,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,work_phone,home_phone,...,Age,birth_month,birth_week,ages_employed,employ_month,employ_week,ages_unemployed,unemploy_month,unemploy_week,income_family
0,M,Y,N,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,0,1,...,60,1.0,1.0,0,0.0,0.0,60,1.0,1.0,56250.0
1,F,N,Y,135000.0,State servant,Higher education,Married,House / apartment,0,1,...,51,8.0,1.0,23,1.0,2.0,28,7.0,2.0,67500.0
2,F,N,Y,69372.0,Working,Secondary / secondary special,Married,House / apartment,1,1,...,43,1.0,1.0,0,7.0,3.0,42,6.0,2.0,34686.0
3,M,Y,N,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,1,0,...,52,6.0,0.0,6,0.0,1.0,45,5.0,3.0,56250.0
4,F,Y,Y,225000.0,State servant,Higher education,Married,House / apartment,1,0,...,48,6.0,2.0,25,0.0,0.0,23,5.0,1.0,112500.0


In [10]:
df_train.describe()

,income_total,work_phone,home_phone,email,family_size,begin_month,credit,days_unemployed,income_unemployed,Age,birth_month,birth_week,ages_employed,employ_month,employ_week,ages_unemployed,unemploy_month,unemploy_week,income_family
count,2.481700e+04,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.00000,24817.000000,24817.000000
mean,1.868744e+05,0.225249,0.294274,0.091510,2.199057,26.054318,1.514325,13753.015514,15.516610,43.194907,5.469920,1.497159,5.608978,4.580932,1.261071,37.174235,5.42620,1.514889,100744.732110
std,1.008656e+05,0.417754,0.455725,0.288339,0.919132,16.532853,0.704571,4867.849160,10.239055,11.523499,3.455598,1.117819,6.429011,3.724893,1.170015,13.337850,3.41686,1.125090,72544.069854
min,2.700000e+04,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,6547.000000,1.268406,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000,0.00000,0.000000,5625.000000
25%,1.215000e+05,0.000000,0.000000,0.000000,2.000000,12.000000,1.000000,9671.000000,8.573070,34.000000,2.000000,0.000000,1.000000,1.000000,0.000000,26.000000,2.00000,0.000000,56250.000000
50%,1.575000e+05,0.000000,0.000000,0.000000,2.000000,24.000000,2.000000,12689.000000,13.401499,42.000000,5.000000,2.000000,4.000000,4.000000,1.000000,34.000000,5.00000,2.000000,78750.000000
75%,2.250000e+05,0.000000,1.000000,0.000000,3.000000,39.000000,2.000000,16952.000000,19.822541,53.000000,9.000000,2.000000,8.000000,8.000000,2.000000,46.000000,8.00000,3.000000,123750.000000
max,1.575000e+06,1.000000,1.000000,1.000000,20.000000,60.000000,2.000000,25152.000000,205.533081,68.000000,11.000000,3.000000,43.000000,11.000000,3.000000,68.000000,11.00000,3.000000,900000.000000


## Data modeling

### Data Preprocessing for construct model

1) Encoding of categorical features

In [11]:
num_col = df_train.dtypes[df_train.dtypes != "object"].index.tolist()
num_col.remove('credit')

cat_col = df_train.dtypes[df_train.dtypes == "object"].index.tolist()

In [12]:
num_col

['income_total',
 'work_phone',
 'home_phone',
 'email',
 'family_size',
 'begin_month',
 'days_unemployed',
 'income_unemployed',
 'Age',
 'birth_month',
 'birth_week',
 'ages_employed',
 'employ_month',
 'employ_week',
 'ages_unemployed',
 'unemploy_month',
 'unemploy_week',
 'income_family']

In [13]:
cat_col

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occup_type',
 'CODE']

In [14]:
# StandardScaler
scaler = StandardScaler()
df_train[num_col] = scaler.fit_transform(df_train[num_col])
df_test[num_col] = scaler.fit_transform(df_test[num_col])

In [15]:
# OrdinalEncoding
Encoder = OrdinalEncoder()
df_train[cat_col] = Encoder.fit_transform(df_train[cat_col], df_train['credit'])
df_test[cat_col] = Encoder.fit_transform(df_test[cat_col])


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24817 entries, 0 to 24816
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             24817 non-null  float64
 1   car                24817 non-null  float64
 2   reality            24817 non-null  float64
 3   income_total       24817 non-null  float64
 4   income_type        24817 non-null  float64
 5   edu_type           24817 non-null  float64
 6   family_type        24817 non-null  float64
 7   house_type         24817 non-null  float64
 8   work_phone         24817 non-null  float64
 9   home_phone         24817 non-null  float64
 10  email              24817 non-null  float64
 11  occup_type         24817 non-null  float64
 12  family_size        24817 non-null  float64
 13  begin_month        24817 non-null  float64
 14  credit             24817 non-null  float64
 15  CODE               24817 non-null  float64
 16  days_unemployed    248

3) Data Split ⇒ We need to split with stratified split

In [17]:
X_train, X_val, y_train, y_val = train_test_split(df_train.loc[:,df_train.columns != 'credit'], df_train['credit'], test_size = 0.2)

In [18]:
X_train.head()

,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,work_phone,home_phone,...,Age,birth_month,birth_week,ages_employed,employ_month,employ_week,ages_unemployed,unemploy_month,unemploy_week,income_family
4323,1.0,0.0,1.0,-0.291229,0.0,4.0,1.0,1.0,-0.539200,-0.64574,...,-1.231847,0.732183,0.449850,0.527467,0.649446,-0.223139,-1.362634,-1.588098,-1.346488,-0.665054
5532,1.0,0.0,1.0,0.154918,0.0,4.0,1.0,1.0,-0.539200,1.54861,...,-0.450819,-0.714773,-1.339385,-0.872466,1.186385,1.486274,-0.013063,0.167935,-0.457652,-0.458279
3792,1.0,0.0,0.0,-0.960450,4.0,4.0,1.0,1.0,1.854598,-0.64574,...,-1.058286,1.600357,0.449850,-0.250273,1.186385,-1.077846,-0.837801,-1.295426,0.431184,-0.768441
4289,1.0,1.0,0.0,-0.201999,0.0,1.0,1.0,1.0,-0.539200,-0.64574,...,-0.884724,-1.004165,0.449850,-0.405821,-0.424432,1.486274,-0.537896,1.338625,1.320020,-0.814966
17153,1.0,0.0,0.0,0.154918,4.0,4.0,1.0,5.0,-0.539200,-0.64574,...,-1.145066,-0.135991,-1.339385,-0.561369,-1.229841,0.631568,-0.762825,-0.417409,0.431184,-0.690901


In [19]:
y_train

4323     2.0
5532     2.0
3792     1.0
4289     0.0
17153    2.0
        ... 
14875    2.0
23050    1.0
13576    2.0
13236    2.0
21114    2.0
Name: credit, Length: 19853, dtype: float64

### Modeling

- Optuna

In [20]:
def objective(trial: Trial) -> float:
    params_svc = {
        "random_state": 42,
        'kernel': 'rbf',
        'C' : trial.suggest_loguniform('C', 1e-5,1e+5),
        'gamma' : trial.suggest_loguniform('gamma', 1e-5,1e+5),
        'probability' : True,
    }
    model = sklearn.svm.SVC(**params_svc)
    model.fit(
        X_train,
        y_train
    )

    svc_pred = model.predict_proba(X_val)
    log_score = log_loss(y_val, svc_pred)
    
    return log_score

In [21]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="svc_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2022-12-11 02:52:45,558] A new study created in memory with name: svc_parameter_opt
[I 2022-12-11 03:03:22,269] Trial 0 finished with value: 0.861232921401275 and parameters: {'C': 0.05564180225431373, 'gamma': 32147.193482816965}. Best is trial 0 with value: 0.861232921401275.
[I 2022-12-11 03:19:37,052] Trial 1 finished with value: 0.873859096971485 and parameters: {'C': 208.90047049266641, 'gamma': 9.695826644515218}. Best is trial 0 with value: 0.861232921401275.
[I 2022-12-11 03:22:10,366] Trial 2 finished with value: 0.8874876110511116 and parameters: {'C': 0.0003632339256943143, 'gamma': 0.0003630322466779861}. Best is trial 0 with value: 0.861232921401275.
[I 2022-12-11 03:24:19,334] Trial 3 finished with value: 0.8766201234266752 and parameters: {'C': 3.809220577048033e-05, 'gamma': 4589.458612326462}. Best is trial 0 with value: 0.861232921401275.
[I 2022-12-11 03:42:01,291] Trial 4 finished with value: 0.886845095633952 and parameters: {'C': 10.260065124896771, 'gamma': 1

Best Score: 0.8265253693909881
Best trial: {'C': 2110.6995036049584, 'gamma': 0.001328590390054419}


In [22]:
study.best_params['C']

2110.6995036049584

In [23]:
study.best_params['gamma']

0.001328590390054419

--- 
K Fold

In [24]:
p_optuna = {
    'kernel' : 'rbf',
    "random_state": 42,
    'C' : study.best_params['C'],
    'gamma' : study.best_params['gamma'],
    'probability' : True,
}

In [25]:
def stratified_kfold_svc(p, n_fold, X, y, X_test):
    # Declaring Stratified K-Fold:
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    
    # Initializing arrays:
    svc_oof = np.zeros((X.shape[0], 3))
    svc_preds = np.zeros((X_test.shape[0], 3))

    # Main loop:
    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============")
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        # Model fit:
        model = SVC(**p)
        model.fit(X_train, y_train)

        svc_oof[valid_idx] = model.predict_proba(X_valid) # Infers the class probability of train dataset
        svc_preds += model.predict_proba(X_test) / n_fold # Infers the class probability of test dataset
        print(f"Log Loss Score: {log_loss(y_valid, svc_oof[valid_idx]):.5f}")

    log_score = log_loss(y, svc_oof)
    print(f"\nLog Loss Score: {log_score:.5f}")

    return svc_oof, svc_preds

In [26]:
svc_oof_optuna, svc_preds_optuna = stratified_kfold_svc(p_optuna, 10, df_train.drop("credit", axis=1),df_train['credit'] , df_test)

============ Fold 0 ============
Log Loss Score: 0.82438
============ Fold 1 ============
Log Loss Score: 0.81768
============ Fold 2 ============
Log Loss Score: 0.82422
============ Fold 3 ============
Log Loss Score: 0.81596
============ Fold 4 ============
Log Loss Score: 0.82578
============ Fold 5 ============
Log Loss Score: 0.81956
============ Fold 6 ============
Log Loss Score: 0.82858
============ Fold 7 ============
Log Loss Score: 0.82033
============ Fold 8 ============
Log Loss Score: 0.81610
============ Fold 9 ============
Log Loss Score: 0.81414

Log Loss Score: 0.82068


In [27]:
p_grid = {
    'kernel' : 'rbf',
    "random_state": 42,
    'C' : 10.0,
    'gamma' : 0.1,
    'probability' : True,
}

In [28]:
svc_oof_grid, svc_preds_grid = stratified_kfold_svc(p_grid, 10, df_train.drop("credit", axis=1),df_train['credit'] , df_test)

============ Fold 0 ============
Log Loss Score: 0.80066
============ Fold 1 ============
Log Loss Score: 0.80777
============ Fold 2 ============
Log Loss Score: 0.81434
============ Fold 3 ============
Log Loss Score: 0.80912
============ Fold 4 ============
Log Loss Score: 0.80386
============ Fold 5 ============
Log Loss Score: 0.79438
============ Fold 6 ============
Log Loss Score: 0.80596
============ Fold 7 ============
Log Loss Score: 0.81159
============ Fold 8 ============
Log Loss Score: 0.80121
============ Fold 9 ============
Log Loss Score: 0.80394

Log Loss Score: 0.80528


In [29]:
df_submission = pd.read_csv('/content/drive/MyDrive/ML4AI/sample_submission.csv')

In [30]:
df_submission.set_index("index", inplace=True)
df_submission[:] = svc_preds_grid
df_submission.head(20)

,0,1,2
index,,,
26457,0.110314,0.231292,0.658394
26458,0.112174,0.232995,0.654832
26459,0.105291,0.229017,0.665693
26460,0.112309,0.233742,0.653949
26461,0.110499,0.238094,0.651407
26462,0.110915,0.229888,0.659197
26463,0.112268,0.233777,0.653955
26464,0.109178,0.228957,0.661865
26465,0.108257,0.237276,0.654466


In [31]:
df_submission.to_csv('/content/drive/MyDrive/ML4AI/SVC with code.csv',index=False)